In [4]:
import random
import os
from collections import deque
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import hashlib
import json
import pickle
import csv

# Configuration for 3x3 grid
GRID_SIZE = 3
EXIT_POS = (1, GRID_SIZE - 1)  # Exit at row 1 (0-indexed), rightmost column

class RushHour3x3:
    def __init__(self):
        self.grid_size = GRID_SIZE
        self.exit_pos = EXIT_POS

    def get_difficulty_config_for_base(self, base_puzzle_num):
        """
        Get difficulty configuration based on base puzzle number (1-150).
        Ensures exactly 50 easy, 50 medium, 50 hard base puzzles.
        """
        # Define difficulty levels with (num_blockers, difficulty_name)
        difficulty_configs = {
            "easy": [(2, "easy"), (3, "easy")],
            "medium": [(4, "medium"), (5, "medium")],
            "hard": [(6, "hard"), (7, "hard")]
        }
        
        # Strict distribution: puzzles 1-50 easy, 51-100 medium, 101-150 hard
        if base_puzzle_num <= 50:
            return random.choice(difficulty_configs["easy"])
        elif base_puzzle_num <= 100:
            return random.choice(difficulty_configs["medium"])
        else:
            return random.choice(difficulty_configs["hard"])

    def generate_random_puzzle(self, num_blockers=3, max_attempts=1000):
        """
        Generates a random puzzle with:
          - 1 car 'C' in a random position NOT at the exit.
          - num_blockers 1x1 blockers 'B1', 'B2', ...
          - 1 empty cell ('.')
        Only returns solvable puzzles.
        """
        for attempt in range(max_attempts):
            # All coordinates in grid
            coords = [(r, c) for r in range(self.grid_size) for c in range(self.grid_size)]
            
            # Place car (not at exit)
            available_for_car = [pos for pos in coords if pos != self.exit_pos]
            car_pos = random.choice(available_for_car)
            coords.remove(car_pos)
            
            # Place blockers
            blockers = {}
            actual_blockers_placed = 0
            for i in range(1, num_blockers + 1):
                if not coords:
                    break
                pos = random.choice(coords)
                coords.remove(pos)
                blockers[f'B{i}'] = pos
                actual_blockers_placed += 1
            
            # Build grid
            grid = [['.' for _ in range(self.grid_size)] for _ in range(self.grid_size)]
            r, c = car_pos
            grid[r][c] = 'C'
            for b, pos in blockers.items():
                r, c = pos
                grid[r][c] = b
            
            # Test if solvable and has at least 1 move
            solution = self.bfs_solve(grid)
            if solution is not None and len(solution) > 0:
                return grid
        
        raise Exception(f"Could not generate solvable puzzle after {max_attempts} attempts")

    def grid_to_tuple(self, grid):
        return tuple(tuple(row) for row in grid)

    def tuple_to_grid(self, t):
        return [list(row) for row in t]

    def find_pos(self, grid, label):
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] == label:
                    return (r, c)
        return None

    def is_solved(self, grid):
        """Check if car 'C' is at the exit position"""
        car_pos = self.find_pos(grid, 'C')
        return car_pos == self.exit_pos

    def get_neighbors(self, state):
        """Get all valid neighboring states from current state"""
        grid = self.tuple_to_grid(state)
        neighbors = []
        
        # Try moving each piece (car or blocker) in all 4 directions
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] != '.':  # Found a piece
                    label = grid[r][c]
                    
                    # Try all 4 directions
                    for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:  # up, down, left, right
                        nr, nc = r + dr, c + dc
                        
                        # Check if new position is valid and empty
                        if (0 <= nr < self.grid_size and 0 <= nc < self.grid_size and 
                            grid[nr][nc] == '.'):
                            
                            # Create new state
                            new_grid = [row[:] for row in grid]  # Deep copy
                            new_grid[nr][nc] = label
                            new_grid[r][c] = '.'
                            neighbors.append(self.grid_to_tuple(new_grid))
        
        return neighbors

    def bfs_solve(self, start_grid):
        """
        BFS solver that returns ONE optimal solution.
        Returns None if no solution exists.
        """
        start_state = self.grid_to_tuple(start_grid)
        
        if self.is_solved(start_grid):
            return []  # Already solved
        
        queue = deque([(start_state, [])])
        visited = {start_state}
        
        while queue:
            current_state, path = queue.popleft()
            
            # Get all possible next states
            for next_state in self.get_neighbors(current_state):
                if next_state in visited:
                    continue
                    
                visited.add(next_state)
                new_path = path + [next_state]
                
                # Check if this state is solved
                if self.is_solved(self.tuple_to_grid(next_state)):
                    return new_path
                
                queue.append((next_state, new_path))
        
        return None

    def verify_solution(self, start_grid, solution_path):
        """Verify that a solution path is valid"""
        if not solution_path:
            return self.is_solved(start_grid)
        
        current = self.grid_to_tuple(start_grid)
        
        for i, next_state in enumerate(solution_path):
            # Check that next_state is a valid neighbor of current
            valid_neighbors = self.get_neighbors(current)
            if next_state not in valid_neighbors:
                print(f"Invalid move at step {i+1}")
                return False
            current = next_state
        
        # Check final state is solved
        final_grid = self.tuple_to_grid(current)
        if not self.is_solved(final_grid):
            print("Final state is not solved")
            return False
        
        return True

    def get_grid_hash(self, grid):
        """Generate a hash for a grid to check uniqueness - includes exit position for extra safety"""
        grid_str = str(self.grid_to_tuple(grid))
        exit_str = str(self.exit_pos)
        combined_str = grid_str + exit_str
        return hashlib.sha256(combined_str.encode()).hexdigest()

    # Transformation functions for augmentations
    def rotate_90_clockwise(self, grid):
        """Rotate grid 90 degrees clockwise"""
        return [[grid[self.grid_size-1-j][i] for j in range(self.grid_size)] for i in range(self.grid_size)]

    def flip_horizontal(self, grid):
        """Flip grid horizontally"""
        return [row[::-1] for row in grid]

    def flip_vertical(self, grid):
        """Flip grid vertically"""
        return grid[::-1]

    def apply_transformation(self, grid, transformation):
        """Apply transformation and return new grid with updated exit position (1-indexed for info)"""
        if transformation == "original":
            return grid, (2, 3)  # 1-indexed for 3x3
        elif transformation == "90_rotation":
            new_grid = self.rotate_90_clockwise(grid)
            return new_grid, (3, 2)
        elif transformation == "180_rotation":
            new_grid = self.rotate_90_clockwise(self.rotate_90_clockwise(grid))
            return new_grid, (2, 1)
        elif transformation == "270_rotation":
            new_grid = self.rotate_90_clockwise(self.rotate_90_clockwise(self.rotate_90_clockwise(grid)))
            return new_grid, (1, 2)
        elif transformation == "horizontal_flip":
            new_grid = self.flip_horizontal(grid)
            return new_grid, (2, 1)
        elif transformation == "vertical_flip":
            new_grid = self.flip_vertical(grid)
            return new_grid, (2, 3)

    def _internal_exit_for_transform(self, transform_name):
        """Return 0-indexed internal exit position for a transform name."""
        if transform_name == "original":
            return (1, 2)
        elif transform_name == "90_rotation":
            return (2, 1)
        elif transform_name == "180_rotation":
            return (1, 0)
        elif transform_name == "270_rotation":
            return (0, 1)
        elif transform_name == "horizontal_flip":
            return (1, 0)
        elif transform_name == "vertical_flip":
            return (1, 2)
        else:
            return (1, 2)

    def draw_grid(self, grid, title="", save_path=None, exit_pos=None):
        """Draw grid and optionally save to file"""
        if exit_pos is None:
            exit_pos = self.exit_pos
        
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.set_xlim(0, self.grid_size)
        ax.set_ylim(0, self.grid_size)
        ax.set_xticks(range(self.grid_size + 1))
        ax.set_yticks(range(self.grid_size + 1))
        
        # Bold grid lines
        for x in range(self.grid_size + 1):
            ax.plot([x, x], [0, self.grid_size], color='black', linewidth=2)
        for y in range(self.grid_size + 1):
            ax.plot([0, self.grid_size], [y, y], color='black', linewidth=2)

        # Don't show title for cleaner appearance
        ax.invert_yaxis()

        # Draw TARGET border (no fill)
        er, ec = exit_pos
        ax.add_patch(
            patches.Rectangle((ec, er), 1, 1,
                              facecolor='none',
                              edgecolor='red', linewidth=6, linestyle='-')
        )

        # Assign colors for pieces
        colors = {'C': 'skyblue'}
        color_palette = ['lightcoral', 'lightgreen', 'plum', 'khaki', 'lightsalmon', 'lightsteelblue', 'orange']
        blocker_count = 0
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                if cell.startswith('B') and cell not in colors:
                    colors[cell] = color_palette[blocker_count % len(color_palette)]
                    blocker_count += 1

        # Draw pieces and coordinates
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                
                # Draw piece if present
                if cell != '.':
                    ax.add_patch(
                        patches.Rectangle((c, r), 1, 1,
                                          facecolor=colors.get(cell, 'white'),
                                          edgecolor='black', linewidth=2)
                    )
                    # Draw piece label in upper part of cell
                    ax.text(c + 0.5, r + 0.25, cell, ha='center', va='center',
                            fontsize=16, fontweight='bold')
                
                # Always draw coordinates in lower part of cell (1-indexed)
                coord_text = f"({r+1},{c+1})"
                ax.text(c + 0.5, r + 0.8, coord_text, ha='center', va='center',
                        fontsize=12, color='black', fontweight='normal')

        # Add TARGET label
        ax.text(ec + 0.85, er + 0.5, 'TARGET', ha='right', va='center',
                fontsize=13, fontweight='bold', color='red',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', 
                         alpha=0.95, edgecolor='red', linewidth=2))

        ax.axis('off')
        
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            plt.close()
        else:
            plt.show()

    def generate_solution_moves(self, start_grid, solution_path):
        """Generate text description of solution moves with 1-indexed coordinate system"""
        moves = []
        current_grid = start_grid
        
        for i, next_state in enumerate(solution_path):
            next_grid = self.tuple_to_grid(next_state)
            
            # Find what moved
            moved_piece = None
            old_pos = None
            new_pos = None
            
            # Find differences between current and next state
            for r in range(self.grid_size):
                for c in range(self.grid_size):
                    if current_grid[r][c] != next_grid[r][c]:
                        if current_grid[r][c] != '.' and next_grid[r][c] == '.':
                            # This piece moved away
                            moved_piece = current_grid[r][c]
                            old_pos = (r, c)
                        elif current_grid[r][c] == '.' and next_grid[r][c] != '.':
                            # This piece moved here
                            new_pos = (r, c)
            
            if moved_piece and old_pos and new_pos:
                # Convert to 1-indexed coordinates to match visual grid
                old_pos_1indexed = (old_pos[0] + 1, old_pos[1] + 1)
                new_pos_1indexed = (new_pos[0] + 1, new_pos[1] + 1)
                moves.append(f"Step {i+1}: {moved_piece} [{old_pos_1indexed[0]},{old_pos_1indexed[1]}] -> [{new_pos_1indexed[0]},{new_pos_1indexed[1]}]")
            
            current_grid = next_grid
        
        return moves

    def generate_puzzle_json(self, grid, exit_pos=None, difficulty="unknown", num_blockers=0, 
                           solution_length=0, transformation="original"):
        """Generate JSON representation of puzzle state with embedded prompt"""
        if exit_pos is None:
            exit_pos = self.exit_pos
        
        # Find car position
        car_pos = self.find_pos(grid, 'C')
        if car_pos is None:
            raise ValueError("No car 'C' found in grid")
        
        # Convert positions to 1-indexed for JSON (matches visual display)
        car_pos_1indexed = [car_pos[0] + 1, car_pos[1] + 1]
        exit_pos_1indexed = [exit_pos[0] + 1, exit_pos[1] + 1]
        
        # Build pieces dictionary
        pieces = {}
        pieces['C'] = {"type": "car", "position": car_pos_1indexed}
        
        # Find all blockers
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                if cell != '.' and cell != 'C':
                    pos_1indexed = [r + 1, c + 1]
                    pieces[cell] = {"type": "blocker", "position": pos_1indexed}
        
        # Generate the prompt for this specific puzzle (NO STRATEGY HINTS)
        prompt = self.generate_puzzle_specific_prompt_no_strategy(grid, exit_pos)
        
        # Create JSON structure
        puzzle_json = {
            "grid": grid,
            "car_position": car_pos_1indexed,
            "exit_position": exit_pos_1indexed,
            "pieces": pieces,
            "puzzle_info": {
                "difficulty": difficulty,
                "num_blockers": num_blockers,
                "total_moves_in_solution": solution_length,
                "transformation": transformation,
                "grid_size": f"{self.grid_size}x{self.grid_size}",
                "coordinate_system": "1-indexed, [row,col] format where [1,1] is top-left"
            },
            "prompt": prompt
        }
        
        return puzzle_json

    def generate_puzzle_specific_prompt_no_strategy(self, grid, exit_pos=None):
        """Generate a prompt WITHOUT strategy hints for true evaluation"""
        if exit_pos is None:
            exit_pos = self.exit_pos
        
        # Find car position
        car_pos = self.find_pos(grid, 'C')
        if car_pos is None:
            raise ValueError("No car 'C' found in grid")
        
        # Convert to 1-indexed coordinates
        car_pos_1indexed = (car_pos[0] + 1, car_pos[1] + 1)
        exit_pos_1indexed = (exit_pos[0] + 1, exit_pos[1] + 1)
        
        # Find blockers and their positions
        blockers = []
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                if cell != '.' and cell != 'C':
                    pos_1indexed = (r + 1, c + 1)
                    blockers.append(f"{cell} at [{pos_1indexed[0]},{pos_1indexed[1]}]")
        
        prompt = f"""Task: Solve this 3x3 Rush Hour puzzle. Move car "C" from position [{car_pos_1indexed[0]},{car_pos_1indexed[1]}] to the TARGET at position [{exit_pos_1indexed[0]},{exit_pos_1indexed[1]}].

Current Pieces:
- Car "C": Position [{car_pos_1indexed[0]},{car_pos_1indexed[1]}]
- Blockers: {', '.join(blockers) if blockers else 'None'}
- TARGET: Position [{exit_pos_1indexed[0]},{exit_pos_1indexed[1]}]

Rules:
- Any piece can move UP, DOWN, LEFT, or RIGHT by exactly one square
- Pieces cannot move outside the 3x3 grid
- Pieces cannot move into occupied squares
- No two pieces can occupy the same square
- Goal: Move car "C" to the TARGET position

Coordinate System: [row,col] format where [1,1] is top-left, [3,3] is bottom-right

Provide your solution as:
<solution>
Step 1: [PIECE] [start_position] -> [end_position]
Step 2: [PIECE] [start_position] -> [end_position]
...
</solution>

Example response format:
<solution>
Step 1: C [2,1] -> [2,2]
Step 2: B1 [1,3] -> [1,2]  
Step 3: C [2,2] -> [1,2]
</solution>
"""

        return prompt

    def generate_master_puzzle_configs(self, num_puzzles=150, random_seed=42):
        """
        Generate master list ensuring exactly 50 easy, 50 medium, 50 hard base puzzles.
        """
        print("🎯 Generating Master Puzzle Configuration List")
        print("=" * 60)
        
        # Set seed for reproducibility
        random.seed(random_seed)
        
        transformations = ["original", "90_rotation", "180_rotation", "270_rotation", "horizontal_flip", "vertical_flip"]
        
        master_configs = []
        seen_hashes = set()
        
        # Track difficulty distribution
        difficulty_counts = {"easy": 0, "medium": 0, "hard": 0}
        base_puzzle_counts = {"easy": 0, "medium": 0, "hard": 0}
        
        base_puzzle_num = 0
        attempts_total = 0
        
        MAX_ATTEMPTS_PER_PUZZLE = 100
        
        while base_puzzle_num < num_puzzles and attempts_total < num_puzzles * MAX_ATTEMPTS_PER_PUZZLE:
            attempts_total += 1
            
            try:
                base_puzzle_num += 1
                num_blockers, difficulty = self.get_difficulty_config_for_base(base_puzzle_num)
                
                base_puzzle = self.generate_random_puzzle(num_blockers=num_blockers)
                base_puzzle_counts[difficulty] += 1
                
                # Process each transformation
                for transform_name in transformations:
                    transformed_grid, new_exit_pos_1indexed = self.apply_transformation(base_puzzle, transform_name)
                    internal_exit_pos = self._internal_exit_for_transform(transform_name)
                    
                    # Temporarily set exit for solving
                    old_exit_pos = self.exit_pos
                    self.exit_pos = internal_exit_pos
                    
                    try:
                        # Check solvability - only get one solution
                        solution = self.bfs_solve(transformed_grid)
                        if solution is None:
                            continue
                        
                        # Check uniqueness
                        grid_hash = self.get_grid_hash(transformed_grid)
                        if grid_hash in seen_hashes:
                            continue
                        
                        # Accept this configuration
                        seen_hashes.add(grid_hash)
                        difficulty_counts[difficulty] += 1
                        
                        # Create configuration record
                        config = {
                            'config_id': len(master_configs) + 1,
                            'base_puzzle_num': base_puzzle_num,
                            'transformation': transform_name,
                            'grid': transformed_grid,
                            'exit_pos_internal': internal_exit_pos,
                            'exit_pos_display': new_exit_pos_1indexed,
                            'difficulty': difficulty,
                            'num_blockers': num_blockers,
                            'grid_hash': grid_hash,
                            'solution_length': len(solution),
                            'solution': solution
                        }
                        
                        master_configs.append(config)
                        
                        if len(master_configs) % 50 == 0:
                            print(f"✓ Generated {len(master_configs)} configurations...")
                            print(f"   Base puzzles: Easy={base_puzzle_counts['easy']}, Medium={base_puzzle_counts['medium']}, Hard={base_puzzle_counts['hard']}")
                            print(f"   Total configs: Easy={difficulty_counts['easy']}, Medium={difficulty_counts['medium']}, Hard={difficulty_counts['hard']}")
                    
                    finally:
                        self.exit_pos = old_exit_pos
            
            except Exception as e:
                base_puzzle_num -= 1  # Retry this puzzle number
                continue
        
        print(f"\n🎉 Master configuration generation complete!")
        print(f"Generated {len(master_configs)} unique configurations")
        print(f"Base puzzles: Easy={base_puzzle_counts['easy']}, Medium={base_puzzle_counts['medium']}, Hard={base_puzzle_counts['hard']}")
        print(f"Total configs: Easy={difficulty_counts['easy']}, Medium={difficulty_counts['medium']}, Hard={difficulty_counts['hard']}")
        print(f"Unique hashes verified: {len(seen_hashes)}")
        
        # Save master configuration list
        master_file = "data/3x3/master_puzzle_configs.json"
        os.makedirs("data/3x3", exist_ok=True)
        
        # Convert to JSON-serializable format
        json_configs = []
        for config in master_configs:
            json_config = config.copy()
            json_config['solution'] = [list(sol) for sol in config['solution']]
            json_configs.append(json_config)
        
        with open(master_file, 'w') as f:
            json.dump(json_configs, f, indent=2)
        
        # Also save as pickle
        with open("data/3x3/master_puzzle_configs.pkl", 'wb') as f:
            pickle.dump(master_configs, f)
        
        print(f"Master configuration saved to: {master_file}")
        
        # Generate CSV documentation
        self.generate_csv_documentation(master_configs, "data/3x3/puzzle_catalog.csv")
        
        return master_configs

    def generate_csv_documentation(self, master_configs, csv_path):
        """Generate CSV file documenting all puzzles with their difficulty labels"""
        with open(csv_path, 'w', newline='') as csvfile:
            fieldnames = ['puzzle_id', 'folder_name', 'base_puzzle', 'transformation', 
                         'difficulty', 'num_blockers', 'solution_length', 'exit_position']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for config in master_configs:
                base_num = config['base_puzzle_num']
                transform = config['transformation']
                
                if transform == "original":
                    folder_name = f"puzzle{base_num}"
                else:
                    folder_name = f"puzzle{base_num}_{transform}"
                
                writer.writerow({
                    'puzzle_id': config['config_id'],
                    'folder_name': folder_name,
                    'base_puzzle': base_num,
                    'transformation': transform,
                    'difficulty': config['difficulty'],
                    'num_blockers': config['num_blockers'],
                    'solution_length': config['solution_length'],
                    'exit_position': f"[{config['exit_pos_display'][0]},{config['exit_pos_display'][1]}]"
                })
        
        print(f"CSV documentation saved to: {csv_path}")
        
        # Print summary statistics
        difficulty_stats = {"easy": 0, "medium": 0, "hard": 0}
        for config in master_configs:
            difficulty_stats[config['difficulty']] += 1
        
        print("\nDifficulty Distribution in CSV:")
        print(f"  Easy: {difficulty_stats['easy']} puzzles")
        print(f"  Medium: {difficulty_stats['medium']} puzzles")
        print(f"  Hard: {difficulty_stats['hard']} puzzles")
        print(f"  Total: {sum(difficulty_stats.values())} puzzles")

    def create_dataset_from_master_configs(self, master_configs, base_folder="data/3x3"):
        """Create dataset from pre-generated master configuration list."""
        print("\n🎯 Creating Dataset from Master Configurations")
        print("=" * 60)
        
        if not os.path.exists(base_folder):
            os.makedirs(base_folder)
        
        processed_count = 0
        
        for config in master_configs:
            try:
                # Extract configuration data
                config_id = config['config_id']
                base_num = config['base_puzzle_num']
                transform_name = config['transformation']
                grid = config['grid']
                internal_exit_pos = config['exit_pos_internal']
                display_exit_pos = config['exit_pos_display']
                difficulty = config['difficulty']
                num_blockers = config['num_blockers']
                solution = config['solution']
                
                # Set appropriate exit position
                old_exit_pos = self.exit_pos
                self.exit_pos = internal_exit_pos
                
                try:
                    # Create folder name
                    if transform_name == "original":
                        folder_name = f"puzzle{base_num}"
                    else:
                        folder_name = f"puzzle{base_num}_{transform_name}"
                    
                    puzzle_folder = os.path.join(base_folder, folder_name)
                    if not os.path.exists(puzzle_folder):
                        os.makedirs(puzzle_folder)
                    
                    # Generate and save puzzle-specific prompt (NO STRATEGY)
                    puzzle_prompt = self.generate_puzzle_specific_prompt_no_strategy(grid, internal_exit_pos)
                    with open(os.path.join(puzzle_folder, "prompt.txt"), 'w') as f:
                        f.write(puzzle_prompt)
                    
                    # Generate and save JSON representation
                    puzzle_json = self.generate_puzzle_json(
                        grid=grid,
                        exit_pos=internal_exit_pos,
                        difficulty=difficulty,
                        num_blockers=num_blockers,
                        solution_length=len(solution),
                        transformation=transform_name
                    )
                    with open(os.path.join(puzzle_folder, "puzzle_state.json"), 'w') as f:
                        json.dump(puzzle_json, f, indent=2)
                    
                    # Save initial state image
                    self.draw_grid(grid, save_path=os.path.join(puzzle_folder, "initial_state.png"), exit_pos=internal_exit_pos)
                    
                    # Save final solved state image
                    if solution:
                        final_state = self.tuple_to_grid(solution[-1])
                        self.draw_grid(final_state, save_path=os.path.join(puzzle_folder, "final_solved_state.png"), exit_pos=internal_exit_pos)
                    
                    # Generate solution text (SINGLE SOLUTION ONLY)
                    moves = self.generate_solution_moves(grid, solution)
                    
                    solution_file = os.path.join(puzzle_folder, "solution.txt")
                    with open(solution_file, 'w') as f:
                        f.write(f"Puzzle: {folder_name}\n")
                        f.write(f"Configuration ID: {config_id}\n")
                        f.write(f"Total moves: {len(solution)}\n")
                        f.write(f"Exit position: [{display_exit_pos[0]},{display_exit_pos[1]}]\n")
                        f.write(f"Transformation: {transform_name}\n")
                        f.write(f"Difficulty: {difficulty} ({num_blockers} blockers)\n")
                        f.write("Coordinate system: [row,col] where [1,1] is top-left\n\n")
                        
                        f.write("SOLUTION:\n")
                        f.write("=" * 40 + "\n")
                        for move in moves:
                            f.write(move + "\n")
                    
                    processed_count += 1
                    
                    if processed_count % 50 == 0:
                        print(f"✓ Processed {processed_count}/{len(master_configs)} configurations...")
                
                finally:
                    self.exit_pos = old_exit_pos
            
            except Exception as e:
                print(f"Error processing configuration {config.get('config_id', 'unknown')}: {e}")
                continue
        
        print(f"\n🎉 Dataset creation complete!")
        print(f"Processed {processed_count}/{len(master_configs)} configurations")
        print(f"Dataset saved in '{base_folder}' folder")
        
        return processed_count

    def load_master_configs(self, filepath="data/3x3/master_puzzle_configs.pkl"):
        """Load master configuration list from file"""
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except FileNotFoundError:
            print(f"Master config file not found: {filepath}")
            return None

    def create_full_dataset(self, num_puzzles=150, random_seed=42):
        """
        Complete workflow: Generate master configs then create dataset.
        Ensures exactly 50 easy, 50 medium, 50 hard base puzzles.
        """
        print("🚀 Starting Full 3x3 Rush Hour Dataset Generation")
        print("=" * 60)
        print(f"Target: {num_puzzles} base puzzles × 6 transformations = {num_puzzles * 6} total puzzles")
        print(f"Distribution: 50 easy + 50 medium + 50 hard = 150 base puzzles")
        print(f"Random seed: {random_seed} (for reproducibility)")
        print()
        
        # CELL 1: Generate master configuration list
        master_configs = self.generate_master_puzzle_configs(num_puzzles, random_seed)
        
        # CELL 2: Create dataset from master configurations
        processed_count = self.create_dataset_from_master_configs(master_configs)
        
        print(f"\n✅ COMPLETE: Generated {len(master_configs)} unique puzzle configurations")
        print(f"✅ COMPLETE: Processed {processed_count} puzzle files")
        print("\nDataset features:")
        print("- ✅ Reproducible generation using fixed random seed")
        print(f"- ✅ All {len(master_configs)} configurations guaranteed unique")
        print("- ✅ Single optimal solution per puzzle")
        print("- ✅ No strategy hints in prompts (true evaluation)")
        print("- ✅ CSV catalog with difficulty labels")
        print("- ✅ Exactly 300 easy, 300 medium, 300 hard puzzles")

    def test_solver(self):
        """Test the BFS solver with known cases"""
        print("Testing 3x3 BFS Solver...")
        
        # Test 1: Already solved puzzle
        solved_grid = [['B1', '.', '.'], 
                       ['B2', '.', 'C'], 
                       ['.', 'B3', '.']]
        solution = self.bfs_solve(solved_grid)
        assert solution == [], f"Test 1 failed: {solution}"
        print("✓ Test 1 passed: Already solved puzzle")
        
        # Test 2: Simple 1-move solution
        simple_grid = [['B1', '.', '.'], 
                       ['B2', 'C', '.'], 
                       ['.', 'B3', '.']]
        solution = self.bfs_solve(simple_grid)
        assert solution is not None and len(solution) == 1, f"Test 2 failed: {solution}"
        assert self.verify_solution(simple_grid, solution), "Test 2 verification failed"
        print("✓ Test 2 passed: 1-move solution")
        
        # Test 3: Generate and solve random puzzle
        random.seed(42)
        test_grid = self.generate_random_puzzle(num_blockers=4)
        solution = self.bfs_solve(test_grid)
        assert solution is not None, "Test 3 failed: No solution found"
        assert self.verify_solution(test_grid, solution), "Test 3 verification failed"
        print(f"✓ Test 3 passed: Random puzzle solved in {len(solution)} moves")
        
        print("All 3x3 BFS solver tests passed! ✅\n")

# ----- Main Execution -----

if __name__ == "__main__":
    print("Creating 3x3 Rush Hour Dataset with Fixed Difficulty Distribution")
    print("This ensures exactly 50 easy, 50 medium, 50 hard base puzzles.")
    print()
    
    game = RushHour3x3()
    
    # Verify BFS solver first
    game.test_solver()
    
    # Generate fresh dataset with proper distribution
    game.create_full_dataset(num_puzzles=150, random_seed=42)
    
    # Option 2: Load existing master configs and regenerate dataset
    # master_configs = game.load_master_configs()
    # if master_configs:
    #     game.create_dataset_from_master_configs(master_configs)
    # else:
    #     print("No existing master configs found. Run Option 1 first.")

Creating 3x3 Rush Hour Dataset with Fixed Difficulty Distribution
This ensures exactly 50 easy, 50 medium, 50 hard base puzzles.

Testing 3x3 BFS Solver...
✓ Test 1 passed: Already solved puzzle
✓ Test 2 passed: 1-move solution
✓ Test 3 passed: Random puzzle solved in 2 moves
All 3x3 BFS solver tests passed! ✅

🚀 Starting Full 3x3 Rush Hour Dataset Generation
Target: 150 base puzzles × 6 transformations = 900 total puzzles
Distribution: 50 easy + 50 medium + 50 hard = 150 base puzzles
Random seed: 42 (for reproducibility)

🎯 Generating Master Puzzle Configuration List
✓ Generated 50 configurations...
   Base puzzles: Easy=9, Medium=0, Hard=0
   Total configs: Easy=50, Medium=0, Hard=0
✓ Generated 100 configurations...
   Base puzzles: Easy=17, Medium=0, Hard=0
   Total configs: Easy=100, Medium=0, Hard=0
✓ Generated 150 configurations...
   Base puzzles: Easy=25, Medium=0, Hard=0
   Total configs: Easy=150, Medium=0, Hard=0
✓ Generated 200 configurations...
   Base puzzles: Easy=34, Me